## 트랜스포머 모델
 앞서 다룬 챗봇은 순환 신경망을 기반으로 한 시퀀스 투 시퀀스 모델을 사용해서 만들었다.
 이번절에서는 시퀀스 투 시퀀스 모델 중 많은 사람들이 사용하고 성능이 좋은 최신 모델인 트랜스 포머 모델을 만들어 보자!

 트랜스포머는 구글이 2017년에 소개한 "Attention is all you need"에 나온 모델로 기존의 시퀀스 투 시퀀스의 인코더 디코더 구조를 가지고 있지만 CNN이나 RNN과 다르게, 어텐션 구조만으로 전체 모델을 만들었다.

 연산이 RNN에 비해 간단함에도 좋은 성능을 보여주고 있다.

In [11]:
import tensorflow as tf
import numpy as np
import sys
# !pip install konlpy

from konlpy.tag import Twitter
import pandas as pd
import tensorflow as tf
import enum
import os
import re
import json
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
sys.path.append('/content/drive/MyDrive/2021-1/AI데이터활용교재개발/자연어처리(텐서플로,머신러닝)/code/')
from preprocess import *

### 모델 소개
앞서 다룬 순환신경망 모델은 시퀀스 순서에 따른 패턴을 보는 것이 중요했다. 
예를들어 '나는 어제 기분이 좋았어'라는 문장을 시퀀스 투 시퀀스를 거쳐 '기분이 좋다니 저도 좋아요'라는 문장을 생성한다고 해보자. 
순환 신경망의 경우 인코더에서 각 스텝을 거쳐 마지막 스텝의 은닉 상태 벡터에 '기분이 좋다'는 문장의 맥락 정보가 반영되어 디코더에서 응답 문장을 생성했다.

이러한 신환신경망 구조를 통한 맥락 정보 추출은 보통의 경우에는 좋은 성능을 보였지만, 단순히 하나의 벡터에 인ㄴ코더 부분에 해당하는 문장에 대한 모든 정보를 담고 있어 개별 단어와 문장 사이의 관계를 파악하기 어렵고, 문장 길이가 길어질수록 하나의 벡터에 정보를 모두 포함하기에는 부족하다는 단점이 있다.

이러한 단점을 극복한 것이 트랜스 포머 모델이다.

트랜스 포머 모델은 기본적으로는 인코더와 디코더로 구성되며, 입력한 문장에 대한 정보와 디코더에 입력한 문장 정보를 조합해서 디코더 문장 다음에 나올 단어를 생성하는 방법이다.

시퀀스투 시퀀스와 다른 점은 순환 신경망을 이용하지 않고 셀프 어텐션 기법을 사용해 정보를 추출한다는 점이다.

#### 셀프 어텐션
셀프 어텐션은 문장에서 각 단어끼리 얼마나 관계가 있는지를 계산해서 반영하는 방법이다.

셀프 어텐션을 이용하면 문장 안에서 단어들 간의 관계를 측정할 수 있고, 단어를 기준으로 다른 단어들과의 관계 값을 계산할 수 있다.

트랜스포머 모델은 단어끼리의 내적을 통해 어텐션 스코어를 구한다.

##### context vector란?

예를들어 '딥러닝'이라는 단어와 '딥러닝''자연어''처리''아주''좋아요''
의 내적값이
25 25 24.3 24.3 24 이고 -> softmax -> 0.3 0.3 0.15 0.1 로 확률값이 되었을때
이 확률값 * 아까 있던 단어 벡터들 끼리의 곱을 해서 더한것이 context vector

#### 멀티헤드 어텐션
내적 어텐션 구조가 중첩된 형태

##### 스케일 내적 어텐션
기본적인 내적 어텐션의 입력은 세가지 query key value 로 구성된다.
찾고자 하는 단어는 query, 사전에 등록 된 단어는 key, 단어의 뜻은 value

In [ ]:

def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # 쿼리와 키의 외적(?). 이걸 계산하면 attention행렬이 나온다.

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk) # 값이 너무 커지는것을 방지

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  # 마스킹 되는 부분은 수를 빼준다. 그러면 softmax씌우면 0에 가까운 값이 된다.

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

##### 순방향 마스크 어텐션
- 트랜스 포머의 경우 전체 문장이 들어가기 때문에, 위치와 상관없이 모든 단어를 참고해서 예측이 가능하다.(아직 예측하지 않은 부분도!)

- 이거는 틀린 상황이기 때문에, 자신보다 뒤에 있는 단어를 참고하지 않게 하는 방법이 마스크 기법이다.

- 아래는 순방향 마스크를 만드는 방법

In [4]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

Mounted at /content/drive


##### 멀티 헤드 어텐션
- 셀프 어텐션에 대한 정보를 하나만 생성해서 모델에서 추론할 수도 있지만, 어텐션이 핳나가 아닌 여럿에 대한 정보를 줄 수 있다면?!

- 멀티헤드 어텐션은 어텐션 맵을 여럿 만들어 다양한 특징에 대해 어텐션을 볼 수 있게 하는 방법

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = kargs['num_heads']
        self.d_model = kargs['d_model'] # key, query, value 에 대한 차원을 정의하기 위한 파라미터

        assert self.d_model % self.num_heads == 0

        self.depth = self.d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(kargs['d_model'])
        self.wk = tf.keras.layers.Dense(kargs['d_model'])
        self.wv = tf.keras.layers.Dense(kargs['d_model'])

        self.dense = tf.keras.layers.Dense(kargs['d_model'])

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights